# 3. Ingeniería de Características y Selección Estadística Avanzada

En esta fase, transformaremos los hallazgos visuales del EDA en decisiones matemáticas, preparando el dataset para el modelado. A diferencia de enfoques estándar, priorizaremos la **integridad de la distribución original** de los datos, evitando técnicas de sobremuestreo sintético que puedan alterar la realidad del negocio.

## Objetivos Técnicos del Notebook:

1.  **Saneamiento y Reducción de Redundancia:**
    * Ejecución de la "Lista de Eliminación" definida en el EDA (borrado de variables con correlación > 0.90 y variables huérfanas como `torque_rpm`).

2.  **Escalado:**
    * Escalado estandar para evitar darle un peso mayor a alguna caracteristica.

3.  **Selección Estadística de Características (Feature Selection):**
    * Aplicaremos pruebas estadísticas de dependencia no lineal (**Mutual Information Classification**) para rankear matemáticamente las variables.
    * Seleccionaremos el subconjunto óptimo de columnas que maximizan la señal de información con el target `claim_status`.

4.  **Codificación (Encoding):**
    * Transformación de variables categóricas a numéricas. Priorizaremos *One-Hot Encoding* para cardinalidades bajas y *Frequency Encoding* para altas.

In [1]:
# importaciones
import os
from pathlib import Path

import pandas as pd
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder

In [2]:
# Routes
DATA_PATH = Path("../data")
PROCESSED_DATA_PATH = DATA_PATH / "processed"
CLEANED_DATA_PATH = DATA_PATH / "cleaned"

In [3]:
# Load dataset
try:
    df = pd.read_csv(PROCESSED_DATA_PATH / "train_folds.csv", sep=",")
    print(f"Dataset loaded successfully. Dimensions: {df.shape}")
except FileNotFoundError:
    print("Error! The file does not exist. Check the path.")

# Initial overview
df.head()

Dataset loaded successfully. Dimensions: (45358, 41)


,subscription_length,vehicle_age,customer_age,region_density,segment,model,fuel_type,airbags,is_esc,is_adjustable_steering,...,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,torque_nm,torque_rpm,power_bhp,power_rpm,claim_status,kfold
0,5.7,0.2,48,8794,C2,M4,Diesel,6,1,1,...,0,1,1,3,250.0,2750.0,113.45,4000.0,0,2
1,0.7,0.0,44,73430,A,M1,CNG,2,0,0,...,0,0,1,0,60.0,3500.0,40.36,6000.0,0,1
2,3.8,1.6,38,6112,C2,M4,Diesel,6,1,1,...,0,1,1,3,250.0,2750.0,113.45,4000.0,0,3
3,1.1,1.4,52,8794,C2,M4,Diesel,6,1,1,...,0,1,1,3,250.0,2750.0,113.45,4000.0,0,3
4,10.1,0.4,52,8794,B2,M6,Petrol,2,0,1,...,1,1,1,2,113.0,4400.0,88.50,6000.0,0,2


In [4]:
# Feature Selection - Drop Redundant and Low-Variance Columns
cols_to_drop = [
    # High correlation with other features
    "displacement",
    "cylinder",
    "width",
    "turning_radius",
    "torque_rpm",
    "power_rpm",
    # Simiular binary features
    "is_ecw",
    "is_power_door_locks",
    "is_rear_window_washer",
    "is_rear_window_wiper",
    "is_tpms",
    "is_driver_seat_height_adjustable",
    # null variance
    "is_speed_alert",
    # by shap values
    "customer_age",
    "gross_weight",
    "is_front_fog_lights",
    "steering_type",
    "length",
]

print(f"Dimensions before reduction: {df.shape}")
df_clean = df.drop(
    columns=[c for c in cols_to_drop if c in df.columns], errors="ignore"
)
print(f"Dimensions after reduction: {df_clean.shape}")
df_clean.head()

Dimensions before reduction: (45358, 41)
Dimensions after reduction: (45358, 23)


,subscription_length,vehicle_age,region_density,segment,model,fuel_type,airbags,is_esc,is_adjustable_steering,is_parking_sensors,...,is_rear_window_defogger,is_brake_assist,is_central_locking,is_power_steering,is_day_night_rear_view_mirror,ncap_rating,torque_nm,power_bhp,claim_status,kfold
0,5.7,0.2,8794,C2,M4,Diesel,6,1,1,1,...,1,1,1,1,0,3,250.0,113.45,0,2
1,0.7,0.0,73430,A,M1,CNG,2,0,0,1,...,0,0,0,1,0,0,60.0,40.36,0,1
2,3.8,1.6,6112,C2,M4,Diesel,6,1,1,1,...,1,1,1,1,0,3,250.0,113.45,0,3
3,1.1,1.4,8794,C2,M4,Diesel,6,1,1,1,...,1,1,1,1,0,3,250.0,113.45,0,3
4,10.1,0.4,8794,B2,M6,Petrol,2,0,1,1,...,0,1,1,1,1,2,113.0,88.50,0,2


In [5]:
# Standar scale numerical features

continuous_vars = [
    "subscription_length",
    "vehicle_age",
    "power_bhp",
    "torque_nm",
    "region_density",
]

scaler = MinMaxScaler()
df_clean[continuous_vars] = scaler.fit_transform(df_clean[continuous_vars])
df_clean.head()

,subscription_length,vehicle_age,region_density,segment,model,fuel_type,airbags,is_esc,is_adjustable_steering,is_parking_sensors,...,is_rear_window_defogger,is_brake_assist,is_central_locking,is_power_steering,is_day_night_rear_view_mirror,ncap_rating,torque_nm,power_bhp,claim_status,kfold
0,0.407143,0.01,0.116270,C2,M4,Diesel,6,1,1,1,...,1,1,1,1,0,3,1.000000,0.937051,0,2
1,0.050000,0.00,1.000000,A,M1,CNG,2,0,0,1,...,0,0,0,1,0,0,0.000000,0.000000,0,1
2,0.271429,0.08,0.079601,C2,M4,Diesel,6,1,1,1,...,1,1,1,1,0,3,1.000000,0.937051,0,3
3,0.078571,0.07,0.116270,C2,M4,Diesel,6,1,1,1,...,1,1,1,1,0,3,1.000000,0.937051,0,3
4,0.721429,0.02,0.116270,B2,M6,Petrol,2,0,1,1,...,0,1,1,1,1,2,0.278947,0.617179,0,2


In [6]:
df_clean.dtypes

subscription_length              float64
vehicle_age                      float64
region_density                   float64
segment                              str
model                                str
fuel_type                            str
airbags                            int64
is_esc                             int64
is_adjustable_steering             int64
is_parking_sensors                 int64
is_parking_camera                  int64
rear_brakes_type                     str
transmission_type                    str
is_rear_window_defogger            int64
is_brake_assist                    int64
is_central_locking                 int64
is_power_steering                  int64
is_day_night_rear_view_mirror      int64
ncap_rating                        int64
torque_nm                        float64
power_bhp                        float64
claim_status                       int64
kfold                              int64
dtype: object

In [7]:
# Mutual Information Classification for Feature Selection
X = df_clean.drop(columns=["claim_status", "kfold"])
y = df_clean["claim_status"]

# ordinal encoding for categorical variables
categorical_cols = X.select_dtypes(
    include=["object", "category", "str"]
).columns.tolist()
encoder = OrdinalEncoder()
X[categorical_cols] = encoder.fit_transform(X[categorical_cols])

# Calculate Mutual Information Scores
discrete_features = [True if str(dtype) == "int64" else False for dtype in X.dtypes]
mi_scores = mutual_info_classif(
    X, y, discrete_features=discrete_features, random_state=42
)
mi_series = pd.Series(mi_scores, index=X.columns).sort_values(ascending=False)

# Select top N features
N = 20
print(f"Top {N} features by Mutual Information Score:")
top_features = mi_series.head(N).index.tolist()
print(top_features)
df_final = df_clean[top_features + ["claim_status", "kfold"]]
print(f"Dimensions after feature selection: {df_final.shape}")
df_final.head()

Top 20 features by Mutual Information Score:
['rear_brakes_type', 'transmission_type', 'segment', 'subscription_length', 'vehicle_age', 'power_bhp', 'torque_nm', 'fuel_type', 'region_density', 'model', 'is_adjustable_steering', 'is_parking_sensors', 'is_brake_assist', 'is_day_night_rear_view_mirror', 'airbags', 'ncap_rating', 'is_esc', 'is_central_locking', 'is_rear_window_defogger', 'is_parking_camera']
Dimensions after feature selection: (45358, 22)


,rear_brakes_type,transmission_type,segment,subscription_length,vehicle_age,power_bhp,torque_nm,fuel_type,region_density,model,...,is_brake_assist,is_day_night_rear_view_mirror,airbags,ncap_rating,is_esc,is_central_locking,is_rear_window_defogger,is_parking_camera,claim_status,kfold
0,Disc,Automatic,C2,0.407143,0.01,0.937051,1.000000,Diesel,0.116270,M4,...,1,0,6,3,1,1,1,1,0,2
1,Drum,Manual,A,0.050000,0.00,0.000000,0.000000,CNG,1.000000,M1,...,0,0,2,0,0,0,0,0,0,1
2,Disc,Automatic,C2,0.271429,0.08,0.937051,1.000000,Diesel,0.079601,M4,...,1,0,6,3,1,1,1,1,0,3
3,Disc,Automatic,C2,0.078571,0.07,0.937051,1.000000,Diesel,0.116270,M4,...,1,0,6,3,1,1,1,1,0,3
4,Drum,Manual,B2,0.721429,0.02,0.617179,0.278947,Petrol,0.116270,M6,...,1,1,2,2,0,1,0,0,0,2


In [9]:
importance_df = mi_series.head(N).to_frame(name="MI_Score")

print("--- Importance ranking ---")
display(importance_df)

--- Importance ranking ---


,MI_Score
rear_brakes_type,0.009313
transmission_type,0.008761
segment,0.004211
subscription_length,0.003205
vehicle_age,0.003086
power_bhp,0.002302
torque_nm,0.002239
fuel_type,0.002147
region_density,0.001796
model,0.001334


In [10]:
THRESHOLD = 0.00002

# Filter features based on threshold
selected_features_names = mi_series[mi_series > THRESHOLD].index.tolist()

print(f"Automatically selected variables (Score > {THRESHOLD}):")
print(selected_features_names)
print(f"Automatically discarded variables (Score <= {THRESHOLD}):")
print(mi_series[mi_series <= THRESHOLD].index.tolist())

# Create the base dataset with the selected features + target
df_final = df_final[selected_features_names + ["claim_status", "kfold"]].copy()
df_final.head()

Automatically selected variables (Score > 2e-05):
['rear_brakes_type', 'transmission_type', 'segment', 'subscription_length', 'vehicle_age', 'power_bhp', 'torque_nm', 'fuel_type', 'region_density', 'model', 'is_adjustable_steering', 'is_parking_sensors', 'is_brake_assist']
Automatically discarded variables (Score <= 2e-05):
['is_day_night_rear_view_mirror', 'airbags', 'ncap_rating', 'is_esc', 'is_central_locking', 'is_rear_window_defogger', 'is_parking_camera', 'is_power_steering']


,rear_brakes_type,transmission_type,segment,subscription_length,vehicle_age,power_bhp,torque_nm,fuel_type,region_density,model,is_adjustable_steering,is_parking_sensors,is_brake_assist,claim_status,kfold
0,Disc,Automatic,C2,0.407143,0.01,0.937051,1.000000,Diesel,0.116270,M4,1,1,1,0,2
1,Drum,Manual,A,0.050000,0.00,0.000000,0.000000,CNG,1.000000,M1,0,1,0,0,1
2,Disc,Automatic,C2,0.271429,0.08,0.937051,1.000000,Diesel,0.079601,M4,1,1,1,0,3
3,Disc,Automatic,C2,0.078571,0.07,0.937051,1.000000,Diesel,0.116270,M4,1,1,1,0,3
4,Drum,Manual,B2,0.721429,0.02,0.617179,0.278947,Petrol,0.116270,M6,1,1,1,0,2


In [11]:
# ordinal Encoding for High Cardinality Categorical Features
segment_map = {"A": 0, "B1": 1, "B2": 2, "C1": 3, "C2": 4, "Utility": 5}
df_final["segment"] = df_final["segment"].map(segment_map)

# one-hot encoding for Low Cardinality Categorical Features
nominal_cols = [
    "fuel_type",
    "model",
    # "steering_type",
    "rear_brakes_type",
    "transmission_type",
]
df_final = pd.get_dummies(df_final, columns=nominal_cols, drop_first=True, dtype=int)
print(f"Dimensions after encoding: {df_final.shape}")
df_final.head()

Dimensions after encoding: (45358, 25)


,segment,subscription_length,vehicle_age,power_bhp,torque_nm,region_density,is_adjustable_steering,is_parking_sensors,is_brake_assist,claim_status,...,model_M2,model_M3,model_M4,model_M5,model_M6,model_M7,model_M8,model_M9,rear_brakes_type_Drum,transmission_type_Manual
0,4,0.407143,0.01,0.937051,1.000000,0.116270,1,1,1,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0.050000,0.00,0.000000,0.000000,1.000000,0,1,0,0,...,0,0,0,0,0,0,0,0,1,1
2,4,0.271429,0.08,0.937051,1.000000,0.079601,1,1,1,0,...,0,0,1,0,0,0,0,0,0,0
3,4,0.078571,0.07,0.937051,1.000000,0.116270,1,1,1,0,...,0,0,1,0,0,0,0,0,0,0
4,2,0.721429,0.02,0.617179,0.278947,0.116270,1,1,1,0,...,0,0,0,0,1,0,0,0,1,1


In [12]:
# save cleaned dataset
os.makedirs(CLEANED_DATA_PATH, exist_ok=True)
df_final.to_csv(CLEANED_DATA_PATH / "cleaned_train_folds.csv", index=False)